In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# import sorting
import random
from sklearn.preprocessing import MinMaxScaler
import math
from collections import Counter,OrderedDict

In [2]:
# import ipywidgets as widgets
# !pip install ipywidgets

In [122]:
movie_data=pd.read_csv('movies.csv')
links=pd.read_csv('links.csv')
ratings=pd.read_csv('ratings.csv')
tags=pd.read_csv('tags.csv')

In [124]:
user_rating[1], pred_matrix

(array([0., 0., 0., ..., 0., 0., 0.]),
 array([ 0.20301172, -0.08113113, -0.01858564, ..., -0.17161535,
        -0.10919687,  0.00021891]))

In [4]:
user_rating=np.zeros((len(np.unique(ratings['userId'])),len(np.unique(ratings['movieId']))))
train=np.zeros(user_rating.shape)
test=np.zeros(user_rating.shape)

In [5]:
for i in np.unique(ratings['userId']):
    k=ratings[ratings['userId']==i]
    for j in range(len(k)):
        if (k['movieId'].iloc[j]-1<=9742):
            user_rating[i-1,k['movieId'].iloc[j]-1]=k['rating'].iloc[j]

In [88]:
user_rating, user_rating.shape

(array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 0. , ..., 0. , 0. , 0. ]]), (610, 9724))

In [7]:
# user.shape

In [118]:
# gbest_users = []
# gbest_movies = []
from tqdm import tqdm as tqdm
gbest = None
# gbest_user = None
# gbest_movie = None
class Particle:
    
    def __init__(self, user_dims, movie_dims):
        
        self.error = float('inf')
        
        ####### USERS #######
#         self.curr_users = []
#         self.per_best_user = []
        self.curr_user = np.random.random(user_dims) #(610,k)
        self.per_best_user = self.curr_user
        self.velocity_user = np.random.random(user_dims)
        
        
#         self.curr_movies = []
#         self.velocity = []
#         self.per_best_movies = []

        ####### MOVIES #######
        self.curr_movie = np.random.random(movie_dims) #(9724,k)
        self.per_best_movie = self.curr_movie
        self.velocity_movie = np.random.random(movie_dims)
        
        
#         self.personal_best = np.dot(self.per_best_user, self.per_best_movie.T)
#         self.velocity = (np.random.random(dimens))
        
        
        
        
        
    def __str__(self):
        return "My error is {err}".format(err = self.error)
    
    
    def move(self):
        self.curr_user += self.velocity_user
        self.curr_movie += self.velocity_movie
        
        
def RMSE(prediction, target):
    return np.sqrt(np.mean((prediction-target)**2))


def PSO(iterations, population, c1, c2, result, latent_features):
    
    particles = []
    
    user_dims = (result.shape[0], latent_features)
    movie_dims = (result.shape[1], latent_features)
    print("userDims: ",user_dims)
    print("movie_dims: ",movie_dims)
#     pbar = tqdm(total=iterations)
    for i in range(population):
        particles.append(Particle(user_dims,movie_dims))
    
    error_min = 0.000001
    i = 0
    error = float("inf")
    gbest = Particle(user_dims,movie_dims)
#     gbest = particles[0]
    pred_matrix = None
    # number of epochs
    while i < iterations and gbest.error > error_min:
        
        for particle in tqdm(particles):
#             print(particle.curr_user)
            # result matrix that the particle produces
            particle_res = np.dot(particle.curr_user, particle.curr_movie.T)
#             print(particle_res.shape)
            
            # error
            particle_error = RMSE(particle_res, result)
#             print(particle_error)
            if particle_error < particle.error:
#                 particle.personal_best = particle_res
                particle.per_best_user = particle.curr_user
                particle.per_best_movie = particle.curr_movie
                particle.error = particle_error
                
            if particle_error < gbest.error:
                
                gbest = particle
#                 print("here")
#                 gbest.error = particle.error
#                 gbest.curr_user = particle.curr_user
#                 gbest.curr_movie = particle.curr_movie
#                 gbest.per_best_user = particle.per_best_user
#                 gbest.per_best_movie = particle.per_best_movie
                pred_matrix = particle_res
#                 print(gbest)
                
                
        for particle in particles:
            particle.velocity_user = 0.02*particle.velocity_user + c1*random.random()*(particle.per_best_user - particle.curr_user) + c2*random.random()*(gbest.per_best_user - particle.curr_user)
            particle.velocity_movie = 0.02*particle.velocity_movie + c1*random.random()*(particle.per_best_movie - particle.curr_movie) + c2*random.random()*(gbest.per_best_movie - particle.curr_movie)
            particle.move()
                
            
        print('epoch: {ep}, error {gbest}'.format(ep=i,gbest=gbest.error))
        i += 1
#         pbar.update(1)
            
#     pbar.close()
    return (gbest, pred_matrix)
    
    
# particle = Particle(2,2)
# print(particle.move)
# print(particle.curr_user)
# particle.move()
# print(particle.curr_user)

In [121]:
gbest, pred_matrix = PSO(10,500,2,5,user_rating,latent_features=3)
pred_matrix.shape


userDims:  (610, 3)
movie_dims:  (9724, 3)


  0%|▎                                                                                 | 2/500 [00:00<00:33, 15.08it/s]

epoch: 0, error 0.8758399956223206


  0%|▎                                                                                 | 2/500 [00:00<00:31, 15.79it/s]

epoch: 1, error 0.8682911551491631


...

  0%|▏                                                                                 | 1/500 [00:00<01:03,  7.83it/s]

epoch: 8, error 0.7853202550702199


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:02<00:00,  8.05it/s]


epoch: 9, error 0.7853202550702199


(610, 9724)

In [ ]:
user_rating.shape

In [69]:
# !jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
u